# Capstone Design: Sedimentation Tank Remainder

### Monroe's comments 
(include this in your next submission so I can verify that you've addressed all of these issues.)

* Add the team name under the title.
* Add the names of the students who are writing this report under the team name.
* Add an introduction explaining the problem that you are solving. The goal is to design a sed tank that can be used in many communities. 
* Define the problem and explain how you will solve it.
* Set this up for working in Python. Define the constants that you already know. You can look in the expert inputs file (see the mathcad source files linked below) to see if there are any other constraints you are missing for the sed tank.
* Start with the python code from the sed tank design challenge.

We don't have a minimum depth of the sed tank. Instead you add up the required depths for each of the components to get the total depth.

The operator can't give the intended dimensions of the tank. Your design will provide those dimensions. 
Influent turbidity might be interesting, but we don't currently have a way to use that to improve the sed tank design. Remember that a water treatment plant must be able to treat water with a very wide range of contaminant concentrations.

I've [zipped the Mathcad code](http://designserver.cee.cornell.edu/aguaclaracode/AguaClaraCodeCompact.zip) for your use. Try the SedTank.xmcd inside the ADT designs. I believe you can use mathcad in computers in Carpenter.

For the next submission you will want to have the majority of the sed tank designed. This is a significant challenge. The AIDE team can help specify what level of detail they need to be able to draw the sed tank.

In [1]:
from aide_design.play import*
from aide_design import floc_model as floc
from pytexit import py2tex

Important constraints:
- Physical size of sedimentation tank
- Amount of materials and cost of construction
- Timing of floc settlement in the tank
- Geographic placement of sed tank
- Upflow velocity in the floc blanket
- Capture velocity
- **Design flow rate**
- Plate settler spacing and width
- Minimum depth of the floc blanket
- Head loss through orifices in the effluent manifold

Operator/User Specifications:
- Intended dimensions of tank
- Inflow data (fluid temperature, turbidity, capture velocity)

Major Design Alternatives:
- Vertical flow sedimentation tank
- Horizontal flow sed tank

Project Strategy:
We plan to focus on the process of a vertical flow sedimentation tank based on the lecture notes. According to the specifications given from the operator/user, we will utilize flow equations discussed in class to create functions that will be applied to the outputted fluid from the remained sed tank. 

In [2]:
# We are experimenting with using head loss at the exit of the diffusers as
# a reasonable design constraint for the design of the
# inlet manifold/diffuser system. This head loss in the exit of the diffuser
# will allow higher velocities (and pressure recovery) in the manifold pipe
# and thus will enable use of smaller diameter manifold pipes.
# -----
# We've set a maximum head loss in the sedimentation tank inlet:
headloss_sed_inlet_max = 1 * u.cm

# The manifold and diffuser pipes used in the sedimentation tank have an SDR of 26.
SDR=26

# Pi_jet_plane is used to estimate the maximum energy dissipation rate in a plane jet. 
# A plane jet is uniform in one dimension. A plane jet can be formed by a long narrow slot.
Pi_jet_plane = 0.225

# Design temperature will be helpful to calculate viscosity
T_design = 15*u.degC

# Plant Flowrate:
flow_plant = 60 * u.L/u.s

# Upflow velocity at the top of the floc blanket:
V_sed_up = 1 * u.mm/u.s

# The corrugated plastic sheets used to make the plate settlers:
W_sed = 42 * u.inch
thickness_sed_plate = 2 * u.mm

# The plate settlers are angled 60° from the horizontal:
angle_sed_plate = 60 * u.deg

# The plate setters are spaced 2.5cm apart (this is the perpendicular
# distance between plates, not the horizontal distance between plates):
s_sed_plate = 2.5 * u.cm

# Plate settler capture velocity:
V_sed_capture = 0.12 * u.mm/u.s

# The minimum port flow (from the first port) divided by the maximum port 
# flow (from the last port) for flow division between sedimentation tanks 
# and for flow distribution from the inlet manifold should be at least:
Pi_sed_manifold_flow = 0.8

g = pc.gravity

## Plate Settlers

* Page in lecture notes on various velocities through the sed tank / plate settlers about V_up and V_vert in plate settlers
    * as length of sed tank decreases, effect of lil wasted area at end of plate settlers row becomes larger bc up flow velocity in floc blanket is kept the same
    * so have to make plate settlers longer as sed tank gets shorter
    * Standardization: make all plate settlers 2 foot long, 2.5 cm spacing - if that meets capture velocity requirement, use as standard design
* can calculate height of plates


In [5]:
V_Upflow = 3 * (u.mm/u.s)
V_Capture = 0.2 * (u.mm/u.s)
Spacing_Settlers = 2.5 *(u.cm)
Angle_sed_plate = 60 * (u.deg)
# We are allowed to neglect the effect of the thickness of the plate settlers
Thickness_sed_plate = 0 * (u.mm)

def Len_sed_plate(V_Upflow, V_Capture, Spacing_Settlers, Angle_sed_plate, Thickness_sed_plate):
    Top = Spacing_Settlers * (V_Upflow / V_Capture -1) + Thickness_sed_plate * (V_Upflow / V_Capture)
    Bottom = math.cos(Angle_sed_plate) * math.sin(Angle_sed_plate)
    Len_sed_plate = Top / Bottom
    return Len_sed_plate.to(u.cm)

Len_sed_plate = Len_sed_plate(V_Upflow, V_Capture, Spacing_Settlers, Angle_sed_plate, Thickness_sed_plate)
print('The plate settlers need to be ' + ut.sig(Len_sed_plate.to(u.cm),4) +' long.')

NameError: name 'math' is not defined

## Floc Blankets

We need a function for the length of floc blanket. Assume the height/depth of the floc blanket to be 25cm.

## Inlet Channel

## Inlet Manifolds

Assume manifold pipe is thin-walled with a SDR of 41.
* ratio between smallest and largest velocity for diffusers
* maximum head loss for jet coming out of diffuser, used 
* these ^ can find diameter for pipe to be and design for diffusers


In [4]:
Manifold_SDR = 41

## Inlet Diffusers

We need a function to calculate the length of diffuser.

In [ ]:
# minor loss equation with K=1
V_diffuser_max = (np.sqrt((2 * g * headloss_sed_inlet_max))).to(u.m / u.s)
print('The maximum velocity of the sed tank diffusers is',V_diffuser_max)

# mass conservation
W_diffuser_inner_min = ((V_sed_up / V_diffuser_max) * W_sed).to(u.mm)
print('The minimum width of the sed tank diffusers is',W_diffuser_inner_min)

Diffuser_SDR=26

# Assumed stretch of the PVC pipes as they are heated and molded:
Pi_PVC_stretch = 1.2

# Nominal diameter of the sed tank diffuser
ND_sed_diffuser = 1 * u.inch


#The cross-sectional area of the pipe wall is:
area_PVC = (np.pi/4) * ((pipe.OD(ND_sed_diffuser)**2) 
                          - (pipe.ID_SDR(ND_sed_diffuser,SDR)**2)
                          )

#The thickness of the wall is reduced by the stretch factor:
thickness_sed_diffuser_wall = ((pipe.OD(ND_sed_diffuser) 
                               - pipe.ID_SDR(ND_sed_diffuser,SDR)) 
                              / (2 * Pi_PVC_stretch)
                              )

# From geometry of the rectangular diffuser opening (assuming perfectly square corners) we have:
B_diffuser = ((area_PVC / (2 * thickness_sed_diffuser_wall)) 
                            - W_diffuser
                            ).to(u.cm)

print("Sed diffuser outer length:", B_diffuser)

S_diffuser = B_diffuser - (2 * thickness_sed_diffuser_wall)
print("Sed diffuser inner length:", S_diffuser)

## Jet Reverser

There is no design needed for jet reverser. Take 3" PVC pipe cut in half. Assume the pipe has a SDR of 26.

## Floc Hopper and floc weir

* certain distance Height beneath plate settlers before you get to floc weir
* idea: clear water in this Height (H_clearwater) ~10cm ish
